In [8]:
import numpy as np
import pandas as pd
import time
from sklearn.linear_model import LogisticRegression
import math
import sys
from scipy.stats.stats import pearsonr
import statsmodels.api as sm
from scipy.stats import ttest_ind as ttest

#start timer
start = time.time()

avgbetas = [0,1]
cov_changes = [(0,0),(1,0),(1,-1)]
group_comparisons = 1
sample_sizes = [1000,5000,10000,21754]
number_of_simulations = sys.argv[1]
stratify_level = 5
cov_elims = [0,1]
groups = [(0,1),(1,2),(1,3),(0,2),(0,3),(2,3)]

p1 = [.1378,.25,.15,.1]
p2 = [.0536,.25,.15,.1]
p3 = [.0647,.25,.2,.4]
p4 = [.744,.25,.5,.4]
#############################
#analyses to run
#X'simple_mean_difference'
#X'linear_regression'
#X'simple_covariate_split_stratification'
#X'stratify_propensities_5'
#X'stratify_propensities_10'
#X'stratify_propensities_20'
#X'weight_inverse_regression_5'
#X'weight_inverse_regression_10'
#X'weight_inverse_regression_20'
#############################



for z in range(int(number_of_simulations)):
    corr_diff = []
    final_effect_sizes = []
    for pgroup in range(group_comparisons):
        for N in sample_sizes:
            N = int(N*1.25)
            k = int(p1[pgroup]*N)
            l = int(p2[pgroup]*N)
            m = int(p3[pgroup]*N)
            n = int(p4[pgroup]*N)
            cor12 = np.corrcoef([1] * k + [0] * (N-k),[0] * (N-l) + [1] * l)[0][1]
            cor13 = np.corrcoef([1] * k + [0] * (N-k),[0] * (N-m) + [1] * m)[0][1]
            cor14 = np.corrcoef([1] * k + [0] * (N-k),[0] * (N-n) + [1] * n)[0][1]
            cor23 = np.corrcoef([1] * l + [0] * (N-l),[0] * (N-m) + [1] * m)[0][1]
            cor24 = np.corrcoef([1] * l + [0] * (N-l),[0] * (N-n) + [1] * n)[0][1]
            cor34 = np.corrcoef([1] * m + [0] * (N-m),[0] * (N-n) + [1] * n)[0][1]

            #specify means for your 5 continuous variables
            mean = (0,0,0,0,0)

            #specify the covariance matrix for your 5 continuous variables
            cov = [(1,-.204,.075,-.054,-.015),
                    (-.204,1,-.106,.123,.068),
                    (.075,-.106,1,-.821,-.069),
                    (-.054,.123,-.821,1,.079),
                    (-.015,.068,-.069,.079,1)]


            #simulate the continuous data from a multivariate normal distribution
            v1,v2,v3,v4,v5 = np.random.multivariate_normal(mean,cov,N).T

            #save values in a dataframe
            index = range(N)
            columns = ['v1','v2','v3','v4','v5']
            df = pd.DataFrame(
                {'v1': v1,
                 'v2': v2,
                 'v3': v3,
                 'v4': v4,
                 'v5': v5}, index=index, columns=columns)
            #create empty group variables that we will fill
            df['Group2'] = 0
            df['Group1'] = 0
            df['Group3'] = 0
            df['Group0'] = 0

            #This is an ordinal variable. There are 4 values, 0, 1, 2, and 3. 
            #When we dummy code this, there will be 4 variables to indicate group membership
            #These are the group totals
            #820,71,59,151
            gt2 = int(p1[pgroup]*N)
            gt1 = int(p2[pgroup]*N)
            gt3 = int(p3[pgroup]*N)
            gt0 = int(p4[pgroup]*N)

            #Puts 1's in the dataframe for the appropriate number of groups
            df.loc[df.index < gt2,'Group2'] = 1
            df.loc[(df.index >= gt2)&(df.index < gt1+gt2),'Group1'] = 1
            df.loc[(df.index >= gt1+gt2)&(df.index < gt1+gt2+gt0),'Group0'] = 1
            df.loc[(df.index >= gt1+gt2+gt0)&(df.index < gt1+gt2+gt0+gt3),'Group3'] = 1

            #randomly shuffles the dataframe so not all of group 2 is at the beginning
            df = df.iloc[np.random.permutation(len(df))]
            df = df.reset_index(drop=True)

            #gets rid of variables I don't need
            df = df[['Group2','Group3','Group1','Group0','v1','v2','v3','v4','v5']]

            #within .01, an order of magnitude less
            #these are the ideal correlations between all 9 of the variables
            Cor2 = np.array([1, cor12, cor13, cor14, .551, -.152, .103, -.080, -.024]) #APClass passed test
            Cor3 = np.array([cor12, 1, cor23, cor24, .006, -.023, .021, -.019, -.014]) #APClass no test
            Cor1 = np.array([cor13, cor23, 1, cor34, .011, -.043, .057, -.056, -.028]) #class no pass test
            Cor0 = np.array([cor14, cor24, cor34, 1, -.475, .156, -.125, .105, .042])  #no AP class

            #a penalization factor to minimize the difference between the actual correlations and the ideal correlations
            power = 5

            dfa = np.array(df).T

            # map group number to column index
            group_index = {2: 0,
                           3: 1,
                           1: 2,
                           0: 3}

            def set_group_membership(dfa, y, g0, g1, g2, g3):
                dfa[group_index[0], y] = g0
                dfa[group_index[1], y] = g1
                dfa[group_index[2], y] = g2
                dfa[group_index[3], y] = g3

                return dfa

            def calculate_sum_error(dfa):
                # calculate the corellation coefficient matrix for the whole 2D array
                tmp_corr = np.corrcoef(dfa)

                sm = 0

                # Calculate the difference between the actual correlations and the ideal correlations
                # Add 1 and take it to some power to minimize large correlations
                sm += np.sum(np.power(1 + np.abs(tmp_corr[group_index[0], :] - Cor0), power))
                sm += np.sum(np.power(1 + np.abs(tmp_corr[group_index[1], :] - Cor1), power))
                sm += np.sum(np.power(1 + np.abs(tmp_corr[group_index[2], :] - Cor2), power))
                sm += np.sum(np.power(1 + np.abs(tmp_corr[group_index[3], :] - Cor3), power))
                
                return sm

            def choose_best_group(dfa, y, sum0, sum1, sum2, sum3):
                if (sum0 < sum1) and (sum0 < sum2) and (sum0 < sum3):   # sum0 is smallest
                    return set_group_membership(dfa, y, 1, 0, 0, 0)
                elif (sum1 < sum0) and (sum1 < sum2) and (sum1 < sum3): # sum1 is smallest
                    return set_group_membership(dfa, y, 0, 1, 0, 0)
                elif (sum2 < sum0) and (sum2 < sum1) and (sum2 < sum3): # sum2 is smallest
                    return set_group_membership(dfa, y, 0, 0, 1, 0)
                elif (sum3 < sum0) and (sum3 < sum1) and (sum3 < sum2): # sum3 is smallest
                    return set_group_membership(dfa, y, 0, 0, 0, 1)
                else:
                    # shouldn't ever get here! If we do, it means that there was more
                    # than one minimum with the same same sum of error.
                    raise Exception("More than one minimum sum! " + sum2 + ", " + sum1 + ", " + sum0 + ", " + sum3)

                return dfa

            for y in range(dfa.shape[1]):
                #put the person in Group 0
                dfa = set_group_membership(dfa, y, 1, 0, 0, 0)
                sum0 = calculate_sum_error(dfa)

                #put the person in Group 1
                dfa = set_group_membership(dfa, y, 0, 1, 0, 0)
                sum1 = calculate_sum_error(dfa)

                #put the person in Group 2
                dfa = set_group_membership(dfa, y, 0, 0, 1, 0)
                sum2 = calculate_sum_error(dfa)

                #put the person in Group 3
                dfa = set_group_membership(dfa, y, 0, 0, 0, 1)
                sum3 = calculate_sum_error(dfa)

                dfa = choose_best_group(dfa, y, sum0, sum1, sum2, sum3)

            #we oversampled to get correct group membership
            #this brings sample size back down to what we want
            N = int(N/1.25)
            count_index = {
                0:int(round(p1[pgroup]*N)),#2998
                1:int(round(p2[pgroup]*N)),#1166
                2:int(round(p3[pgroup]*N)),#1407
                3:int(round(p4[pgroup]*N))#16183
            }

            df = pd.DataFrame(dfa.T)
            
            #randomly pull out samples (the right number) for each group
            for group in range(4):
                tmp = df[df[group] == 1].copy()
                tmp['random'] = 0
                tmp['random'] = tmp['random'].apply(lambda x: np.random.normal())
                tmp = tmp.sort_values('random')
                tmp = tmp.reset_index(drop=True)
                tmp = tmp[0:count_index[group]]
                if group == 0:
                    tmp2 = tmp.copy()
                    continue
                pieces = [tmp2,tmp]
                tmp2 = pd.concat(pieces)

            #calculate the correlation matrix difference between desired and simulated  
            df3 = pd.DataFrame(0,index=range(9),columns=['G2','G3','G1','G0'])
            for x in range(9):
                df3.loc[x,'G2'] = np.corrcoef(tmp2[0],tmp2[tmp2.columns[x]])[0][1] - Cor2[x]
                df3.loc[x,'G1'] = np.corrcoef(tmp2[2],tmp2[tmp2.columns[x]])[0][1] - Cor1[x]
                df3.loc[x,'G0'] = np.corrcoef(tmp2[3],tmp2[tmp2.columns[x]])[0][1] - Cor0[x]
                df3.loc[x,'G3'] = np.corrcoef(tmp2[1],tmp2[tmp2.columns[x]])[0][1] - Cor3[x]
            midpoint2 = time.time()
            
            #modify original data based on covariate changes
            tmp = tmp2.copy()
            for cov_change in cov_changes:
                tmp2 = tmp.copy()
                
                if cov_change[0] == 1:
                    tmp2[8] = tmp2[0] + tmp2[8]
                if cov_change[1] == -1:
                    tmp2[8] = tmp2[0] + tmp2[0] + tmp2[8] + [-1] * len(tmp2[8])
                
                #2 different analysis scripts
                #avg beta = 0 runs a separate logistic regression for each group
                #avg beta = 1 averages betas across all group logistic regressions
                

                for avgbeta in avgbetas:
                    if avgbeta == 0:
                        for grouping in groups:
                            for cov_elim in cov_elims:
                                
                                dfa2 = tmp2[(tmp2[grouping[0]]==1)|(tmp2[grouping[1]]==1)].copy()

                                mydf = dfa2.copy()
                                newCorr = pearsonr(mydf[grouping[0]],mydf[4])[0]

                                #this is our dependent variable
                                y = dfa2[grouping[0]]
                                y = np.asarray(y, dtype="|S6")

                                dfagroup1 = tmp2[(tmp2[grouping[0]]==1)]
                                dfagroup2 = tmp2[(tmp2[grouping[1]]==1)]
                                meangroup1 = dfagroup1[4].mean()
                                meangroup2 = dfagroup2[4].mean()
                                mediangroup1 = np.median(dfagroup1[4])
                                mediangroup2 = np.median(dfagroup2[4])
                                stdgroup1 = dfagroup1[4].std(ddof=1)
                                stdgroup2 = dfagroup2[4].std(ddof=1)
                                pooledstd = dfa2[4].std(ddof=1)

                                simpleCohenMeasure = (meangroup1 - meangroup2) / pooledstd

                                #these are our independent variables
                                if cov_elim == 0:
                                    X = np.array(dfa2[[5,6,7,8]])
                                elif cov_elim == 1:
                                    X = np.array(dfa2[[6,7,8]])

                                #initiate the model and fit it
                                model = LogisticRegression()
                                model = model.fit(X, y)
                                probs = model.predict_proba(X)
                                one_prob = probs[:,1]
                                seq = []

                                sep = 100.0 / stratify_level
                                for xy3 in range(stratify_level):
                                    seq.append((xy3+1)*sep)
                                seq = seq[:-1]
    #                             seq = [10,20,25,30,40,50,60,70,80,90]
    #                             stratify_level = 11
                                dec1 = np.percentile(one_prob,seq)
                                dfa2 = np.array(dfa2.T)
                                final_dict_strat = {}

                                ###for simple regression
                                y2 = np.array(tmp2[(tmp2[grouping[0]]==1)|(tmp2[grouping[1]]==1)][4])
                                if cov_elim == 0:
                                    X2 = np.array(tmp2[(tmp2[grouping[0]]==1)|(tmp2[grouping[1]]==1)][[grouping[0],5,6,7,8]])
                                elif cov_elim == 1:
                                    X2 = np.array(tmp2[(tmp2[grouping[0]]==1)|(tmp2[grouping[1]]==1)][[grouping[0],6,7,8]])
                                X2 = sm.add_constant(X2)
                                model = sm.OLS(y2,X2)
                                results = model.fit()

                                linear_regression_result = (results.params[1]) / tmp2[(tmp2[grouping[0]]==1)|(tmp2[grouping[1]]==1)][4].std(ddof=1)

                                ###for stratification on propensities (weighted average)
                                for group_comp in range(len(seq)-1):
                                    final_dict_strat['treatment{}'.format(group_comp+1)] = dfa2[4][(dfa2[grouping[0]]==1)&(one_prob<=dec1[group_comp+1])&(one_prob>dec1[group_comp])]
                                    final_dict_strat['control{}'.format(group_comp+1)] = dfa2[4][(dfa2[grouping[1]]==1)&(one_prob<=dec1[group_comp+1])&(one_prob>dec1[group_comp])]
                                final_dict_strat['treatment{}'.format(0)] = dfa2[4][(dfa2[grouping[0]]==1)&(one_prob<=dec1[0])]
                                final_dict_strat['control{}'.format(0)] = dfa2[4][(dfa2[grouping[1]]==1)&(one_prob<=dec1[0])]
                                final_dict_strat['treatment{}'.format(stratify_level-1)] = dfa2[4][(dfa2[grouping[0]]==1)&(one_prob>dec1[len(seq)-1])]
                                final_dict_strat['control{}'.format(stratify_level-1)] = dfa2[4][(dfa2[grouping[1]]==1)&(one_prob>dec1[len(seq)-1])]

                                effect_sizes = []
                                for x in range(stratify_level):
                                    #mean difference between groups divided by pooled standard deviation
                                    effect_size = (final_dict_strat['treatment{}'.format(x)].mean() - final_dict_strat['control{}'.format(x)].mean())/\
                                                    (np.array(list(final_dict_strat['treatment{}'.format(x)]) + list(final_dict_strat['control{}'.format(x)]))).std(ddof=1)
                                    if effect_size == effect_size:
                                        effect_sizes.append(effect_size)
                                    else:
                                        pass

                                #calculate Cohen's D as the mean of all effect sizes across deciles
                                if np.array(effect_sizes).mean() == np.array(effect_sizes).mean():
                                    CohenD = np.array(effect_sizes).mean()
                                else:
                                    CohenD = None

                                #####
                                final_dict = {}
                                ###for weighted least squares regression
                                test = tmp2[(tmp2[grouping[0]]==1)|(tmp2[grouping[1]]==1)].copy()
                                treatmentw = (1/one_prob[(dfa2[grouping[0]]==1)])/((1/one_prob[(dfa2[grouping[0]]==1)]).sum())
                                controlw = (1/(1-one_prob[(dfa2[grouping[1]]==1)]))/((1/(1-one_prob[(dfa2[grouping[1]]==1)])).sum())
                                test.loc[test[grouping[0]] == 1,'weight'] = treatmentw
                                test.loc[test[grouping[1]] == 1,'weight'] = controlw
                                w = np.array(test['weight'])
                                mod_wls = sm.WLS(y2, X2, weights = w)
                                res_wls = mod_wls.fit()
                                weighted_regression_result = (res_wls.params[1]) / tmp2[4].std(ddof=1)

                                #simpleCohenMeasure
                                #linear_regression_result
                                #weighted_regression_result
                                differences = []
                                for g in range(1,stratify_level-2):
                                    for f in [5,6,7,8]:
                                        group1value = dfa2[f][(dfa2[grouping[0]]==1)&(one_prob<=dec1[g])&(one_prob>dec1[g-1])].mean()
                                        group2value = dfa2[f][(dfa2[grouping[1]]==1)&(one_prob<=dec1[g])&(one_prob>dec1[g-1])].mean()
                                        difference = group1value - group2value
                                        differences.append(difference)
                                #for the first stratum
                                for f in [5,6,7,8]:
                                    group1value = (dfa2[f][(dfa2[grouping[0]]==1)&(one_prob>dec1[stratify_level-2])]).mean()
                                    group2value = (dfa2[f][(dfa2[grouping[1]]==1)&(one_prob>dec1[stratify_level-2])]).mean()
                                    difference = group1value - group2value
                                    differences.append(difference)
                                #for the last stratum
                                for f in [5,6,7,8]:
                                    group1value = (dfa2[f][(dfa2[grouping[0]]==1)&(one_prob<dec1[0])]).mean()
                                    group2value = (dfa2[f][(dfa2[grouping[1]]==1)&(one_prob<dec1[0])]).mean()
                                    difference = group1value - group2value
                                    differences.append(difference)

                                covariate_diff = np.sum(np.abs(differences))
                                differences2 = []
                                ##############ONE PROB###
                                for g in range(1,stratify_level-2):
                                    for f in [5,6,7,8]:
                                        group1value = one_prob[(dfa2[grouping[0]]==1)&(one_prob<=dec1[g])&(one_prob>dec1[g-1])].mean()
                                        group2value = one_prob[(dfa2[grouping[1]]==1)&(one_prob<=dec1[g])&(one_prob>dec1[g-1])].mean()
                                        difference = group1value - group2value
                                        differences2.append(difference)
                                for f in [5,6,7,8]:
                                    group1value = (one_prob[(dfa2[grouping[0]]==1)&(one_prob>dec1[stratify_level-2])]).mean()
                                    group2value = (one_prob[(dfa2[grouping[1]]==1)&(one_prob>dec1[stratify_level-2])]).mean()
                                    difference = group1value - group2value
                                    differences2.append(difference)
                                for f in [5,6,7,8]:
                                    group1value = (one_prob[(dfa2[grouping[0]]==1)&(one_prob<dec1[0])]).mean()
                                    group2value = (one_prob[(dfa2[grouping[1]]==1)&(one_prob<dec1[0])]).mean()
                                    difference = group1value - group2value
                                    differences2.append(difference)

                                propensity_diff = np.sum(np.abs(differences))

                                ###############
                                unbalanced = []
                                for g in range(1,stratify_level-2):
                                    for f in [5,6,7,8]:
                                        group0 = dfa2[f][(dfa2[grouping[0]]==1)&(one_prob<=dec1[g])&(one_prob>dec1[g-1])]
                                        group1 = dfa2[f][(dfa2[grouping[1]]==1)&(one_prob<=dec1[g])&(one_prob>dec1[g-1])]
                                        if len(group0) < 2 or len(group1) < 2:
                                            pass
                                        else:
                                            sig = ttest(group0,group1)[1]
                                            if sig < .05:
                                                unbalanced.append({'group#':g,
                                                                  'variable#':f,
                                                                  'group0size':len(group0),
                                                                  'group1size':len(group1),
                                                                  'p-value':sig})
                                for f in [5,6,7,8]:
                                    g = stratify_level-1
                                    group0 = (dfa2[f][(dfa2[grouping[0]]==1)&(one_prob>dec1[stratify_level-2])])
                                    group1 = (dfa2[f][(dfa2[grouping[1]]==1)&(one_prob>dec1[stratify_level-2])])
                                    if len(group0) < 2 or len(group1) < 2:
                                        pass
                                    else:
                                        sig = ttest(group0,group1)[1]
                                        if sig < .05:
                                            unbalanced.append({'group#':g,
                                                                  'variable#':f,
                                                                  'group0size':len(group0),
                                                                  'group1size':len(group1),
                                                                  'p-value':sig})
                                    #for the last stratum
                                for f in [5,6,7,8]:
                                    g = 0
                                    group0 = (dfa2[f][(dfa2[grouping[0]]==1)&(one_prob<dec1[0])])
                                    group1 = (dfa2[f][(dfa2[grouping[1]]==1)&(one_prob<dec1[0])])
                                    if len(group0) < 2 or len(group1) < 2:
                                        pass
                                    else:
                                        sig = ttest(group0,group1)[1]
                                        if sig < .05:
                                            unbalanced.append({'group#':g,
                                                                  'variable#':f,
                                                                  'group0size':len(group0),
                                                                  'group1size':len(group1),
                                                                  'p-value':sig})
                                ###############

                                ###
                                counting1 = []
                                counting2 = []
                                for x in range(stratify_level):
                                    counting1.append(len(final_dict_strat['treatment{}'.format(x)]))
                                    counting2.append(len(final_dict_strat['control{}'.format(x)]))


                                #Simple Stratification
                                #using binary cutoff points for covariates (greater than median is set to 1, less than median is set to 0)
                                ###############
                                tmp2[15] = tmp2[5].map(lambda x: 1 if x > tmp2[5].median() else 0)
                                tmp2[16] = tmp2[6].map(lambda x: 1 if x > tmp2[6].median() else 0)
                                tmp2[17] = tmp2[7].map(lambda x: 1 if x > tmp2[7].median() else 0)
                                tmp2[18] = tmp2[8].map(lambda x: 1 if x > tmp2[8].median() else 0)

                                tmp2['group'] = tmp2[15].astype(str) + tmp2[16].astype(str) + tmp2[17].astype(str) + tmp2[18].astype(str)

                                groups2 = ['0000','0001','0010','0011','0100','0101','0110','0111','1000','1001','1010','1011','1100','1101','1110','1111']

                                finalresult = 0
                                for group in groups2:
                                    meantreatment = tmp2[(tmp2.group == group)&(tmp2[grouping[0]] == 1)][4].mean()
                                    meancontrol = tmp2[(tmp2.group == group)&(tmp2[grouping[1]] == 1)][4].mean()
                                    temp = tmp2[(tmp2.group == group)&((tmp2[grouping[1]] == 1)|(tmp2[grouping[0]] == 1))].copy()
                                    groupstd = temp[4].std(ddof=1)
                                    result = (meantreatment - meancontrol) / groupstd

                                    groupmembership = len(temp[4])
                                    total = float(len(tmp2[((tmp2[grouping[1]] == 1)|(tmp2[grouping[0]] == 1))][4]))
                                    weight = groupmembership / total

                                    if result == result and weight == weight:
                                        finalresult += result * weight

                                unbalanced2 = []
                                for g in groups2:
                                    for f in [5,6,7,8]:
                                        group0 = tmp2[(tmp2[grouping[0]]==1)&(tmp2.group == g)][f]
                                        group1 = tmp2[(tmp2[grouping[1]]==1)&(tmp2.group == g)][f]
                                        if len(group0) < 2 or len(group1) < 2:
                                            pass
                                        else:
                                            sig = ttest(group0,group1)[1]
                                            if sig < .05:
                                                unbalanced2.append({'group#':g,
                                                                  'variable#':f,
                                                                  'group0size':len(group0),
                                                                  'group1size':len(group1),
                                                                  'p-value':sig})
                                ###############

                                if 'variable#' in pd.DataFrame(unbalanced2).columns:
                                    unbalancedSSvar = pd.DataFrame(unbalanced2)['variable#'].values
                                else:
                                    unbalancedSSvar = None
                                if 'group#' in pd.DataFrame(unbalanced2).columns:
                                    unbalancedSSgroup = pd.DataFrame(unbalanced2)['group#'].values
                                else:
                                    unbalancedSSgroup = None
                                if 'variable#' in pd.DataFrame(unbalanced).columns:
                                    unbalancedCSvar = pd.DataFrame(unbalanced)['variable#'].values
                                else:
                                    unbalancedCSvar = None
                                if 'group#' in pd.DataFrame(unbalanced).columns:
                                    unbalancedCSgroup = pd.DataFrame(unbalanced)['group#'].values
                                else:
                                    unbalancedCSgroup = None



                                #final output
                                final_effect_sizes.append({'Treatment':grouping[0],
                                                           'Control':grouping[1],
                                                           'Proportion':pgroup,
                                                           'N':N,
                                                           'Sim#':z,
                                                           'CovMatrix':tmp2.corr(),
                                                           'CorrDiff':df3.abs().sum().sum(),
                                                           'AvgBeta':avgbeta,
                                                           'CovChange':cov_change,
                                                           'simpleCohenMeasure':simpleCohenMeasure,
                                                           'linear_regression_result':linear_regression_result,
                                                           'weighted_regression_result':weighted_regression_result,
                                                           'stratifiedCohenMeasure':CohenD,
                                                           'meangroup1':meangroup1,
                                                           'meangroup2':meangroup2,
                                                           'stdgroup1':stdgroup1,
                                                           'stdgroup2':stdgroup2,
                                                           'mediangroup1':mediangroup1,
                                                           'mediangroup2':mediangroup2,
                                                           'covariate_diff':covariate_diff,
                                                           'propensity_diff':propensity_diff,
                                                           'counting1':counting1,
                                                           'counting2':counting2,
                                                           'newCorr':newCorr,
                                                           'simplestratification':finalresult,
                                                           'unbalancedSSvar#':unbalancedSSvar,
                                                           'unbalancedSSgroup#':unbalancedSSgroup,
                                                           'unbalancedCSvar#':unbalancedCSvar,
                                                           'unbalancedCSgroup#':unbalancedCSgroup,
                                                           'covariate_elimination':cov_elim
                                                            })
                    else:
                        for cov_elim in cov_elims:
                            coefficient1 = []
                            coefficient2 = []
                            coefficient3 = []
                            coefficient4 = []
                            intercept = []
                            for grouping in groups:

                                dfa2 = tmp2[(tmp2[grouping[0]]==1)|(tmp2[grouping[1]]==1)].copy()

                                #this is our dependent variable
                                y = dfa2[grouping[0]]
                                y = np.asarray(y, dtype="|S6")

                                #these are our independent variables
                                if cov_elim == 0:
                                    X = np.array(dfa2[[5,6,7,8]])
                                elif cov_elim == 1:
                                    X = np.array(dfa2[[6,7,8]])
                                #initiate the model and fit it
                                model = LogisticRegression()
                                model = model.fit(X, y)
                                if cov_elim == 0:
                                    coefficient1.append(model.coef_[0][0])
                                    coefficient2.append(model.coef_[0][1])
                                    coefficient3.append(model.coef_[0][2])
                                    coefficient4.append(model.coef_[0][3])
                                    intercept.append(model.intercept_)
                                elif cov_elim == 1:
                                    coefficient2.append(model.coef_[0][0])
                                    coefficient3.append(model.coef_[0][1])
                                    coefficient4.append(model.coef_[0][2])
                                    intercept.append(model.intercept_)
                            if cov_elim == 0:
                                coefficient_1 = np.array(coefficient1).mean()
                                coefficient_2 = np.array(coefficient2).mean()
                                coefficient_3 = np.array(coefficient3).mean()
                                coefficient_4 = np.array(coefficient4).mean()
                                intercept1 = np.array(intercept).mean()
                            elif cov_elim == 1:
                                coefficient_2 = np.array(coefficient2).mean()
                                coefficient_3 = np.array(coefficient3).mean()
                                coefficient_4 = np.array(coefficient4).mean()
                                intercept1 = np.array(intercept).mean()

                            for grouping in groups:

                                dfa2 = tmp2[(tmp2[grouping[0]]==1)|(tmp2[grouping[1]]==1)].copy()

                                mydf = dfa2.copy()
                                newCorr = pearsonr(mydf[grouping[0]],mydf[4])[0]

                                #this is our dependent variable
                                y = dfa2[grouping[0]]
                                y = np.asarray(y, dtype="|S6")

                                dfagroup1 = tmp2[(tmp2[grouping[0]]==1)]
                                dfagroup2 = tmp2[(tmp2[grouping[1]]==1)]
                                meangroup1 = dfagroup1[4].mean()
                                meangroup2 = dfagroup2[4].mean()
                                mediangroup1 = np.median(dfagroup1[4])
                                mediangroup2 = np.median(dfagroup2[4])
                                stdgroup1 = dfagroup1[4].std(ddof=1)
                                stdgroup2 = dfagroup2[4].std(ddof=1)
                                pooledstd = dfa2[4].std(ddof=1)

                                simpleCohenMeasure = (meangroup1 - meangroup2) / pooledstd

                                #independent variables
                                if cov_elim == 0:
                                    X = np.array(dfa2[[5,6,7,8,]])
                                elif cov_elim == 1:
                                    X = np.array(dfa2[[6,7,8]])

                                #create the model and probabilities
                                if cov_elim == 0:
                                    logs = intercept1 + X[:,0]*coefficient_1 + X[:,1]*coefficient_2 + \
                                                    X[:,2]*coefficient_3 + X[:,3]*coefficient_4
                                elif cov_elim == 1:
                                    logs = intercept1 + X[:,0]*coefficient_2 + \
                                                    X[:,1]*coefficient_3 + X[:,2]*coefficient_4
                                def logToProb(value):
                                    return math.exp(value)/(1+math.exp(value))
                                logToProb = np.vectorize(logToProb)

                                one_prob = logToProb(logs)

                                ######

                                dfagroup1 = tmp2[(tmp2[grouping[0]]==1)]
                                dfagroup2 = tmp2[(tmp2[grouping[1]]==1)]
                                meangroup1 = dfagroup1[4].mean()
                                meangroup2 = dfagroup2[4].mean()
                                mediangroup1 = np.median(dfagroup1[4])
                                mediangroup2 = np.median(dfagroup2[4])
                                stdgroup1 = dfagroup1[4].std(ddof=1)
                                stdgroup2 = dfagroup2[4].std(ddof=1)
                                pooledstd = dfa2[4].std(ddof=1)

                                simpleCohenMeasure = (meangroup1 - meangroup2) / pooledstd

                                #these are our independent variables
                                if cov_elim == 0:
                                    X = np.array(dfa2[[5,6,7,8]])
                                elif cov_elim == 1:
                                    X = np.array(dfa2[[6,7,8]])

                                seq = []

                                sep = 100.0 / stratify_level
                                for xy3 in range(stratify_level):
                                    seq.append((xy3+1)*sep)
                                seq = seq[:-1]
                                dec1 = np.percentile(one_prob,seq)
                                dfa2 = np.array(dfa2.T)
                                final_dict_strat = {}

                                ###for simple regression
                                y2 = np.array(tmp2[(tmp2[grouping[0]]==1)|(tmp2[grouping[1]]==1)][4])
                                if cov_elim == 0:
                                    X2 = np.array(tmp2[(tmp2[grouping[0]]==1)|(tmp2[grouping[1]]==1)][[grouping[0],5,6,7,8]])
                                elif cov_elim == 1:
                                    X2 = np.array(tmp2[(tmp2[grouping[0]]==1)|(tmp2[grouping[1]]==1)][[grouping[0],6,7,8]])
                                X2 = sm.add_constant(X2)
                                model = sm.OLS(y2,X2)
                                results = model.fit()

                                linear_regression_result = (results.params[1]) / tmp2[(tmp2[grouping[0]]==1)|(tmp2[grouping[1]]==1)][4].std(ddof=1)

                                ###for stratification on propensities (weighted average)
                                for group_comp in range(len(seq)-1):
                                    final_dict_strat['treatment{}'.format(group_comp+1)] = dfa2[4][(dfa2[grouping[0]]==1)&(one_prob<=dec1[group_comp+1])&(one_prob>dec1[group_comp])]
                                    final_dict_strat['control{}'.format(group_comp+1)] = dfa2[4][(dfa2[grouping[1]]==1)&(one_prob<=dec1[group_comp+1])&(one_prob>dec1[group_comp])]
                                final_dict_strat['treatment{}'.format(0)] = dfa2[4][(dfa2[grouping[0]]==1)&(one_prob<=dec1[0])]
                                final_dict_strat['control{}'.format(0)] = dfa2[4][(dfa2[grouping[1]]==1)&(one_prob<=dec1[0])]
                                final_dict_strat['treatment{}'.format(stratify_level-1)] = dfa2[4][(dfa2[grouping[0]]==1)&(one_prob>dec1[len(seq)-1])]
                                final_dict_strat['control{}'.format(stratify_level-1)] = dfa2[4][(dfa2[grouping[1]]==1)&(one_prob>dec1[len(seq)-1])]

                                effect_sizes = []
                                for x in range(stratify_level):
                                    #mean difference between groups divided by pooled standard deviation
                                    effect_size = (final_dict_strat['treatment{}'.format(x)].mean() - final_dict_strat['control{}'.format(x)].mean())/\
                                                    (np.array(list(final_dict_strat['treatment{}'.format(x)]) + list(final_dict_strat['control{}'.format(x)]))).std(ddof=1)
                                    if effect_size == effect_size:
                                        effect_sizes.append(effect_size)
                                    else:
                                        pass

                                #calculate Cohen's D as the mean of all effect sizes across deciles
                                if np.array(effect_sizes).mean() == np.array(effect_sizes).mean():
                                    CohenD = np.array(effect_sizes).mean()
                                else:
                                    CohenD = None

                                #####
                                final_dict = {}
                                ###for weighted least squares regression
                                test = tmp2[(tmp2[grouping[0]]==1)|(tmp2[grouping[1]]==1)].copy()
                                treatmentw = (1/one_prob[(dfa2[grouping[0]]==1)])/((1/one_prob[(dfa2[grouping[0]]==1)]).sum())
                                controlw = (1/(1-one_prob[(dfa2[grouping[1]]==1)]))/((1/(1-one_prob[(dfa2[grouping[1]]==1)])).sum())
                                test.loc[test[grouping[0]] == 1,'weight'] = treatmentw
                                test.loc[test[grouping[1]] == 1,'weight'] = controlw
                                w = np.array(test['weight'])
                                mod_wls = sm.WLS(y2, X2, weights = w)
                                res_wls = mod_wls.fit()
                                weighted_regression_result = (results.params[1]) / tmp2[4].std(ddof=1)

                                #simpleCohenMeasure
                                #linear_regression_result
                                #weighted_regression_result
                                differences = []
                                for g in range(1,stratify_level-2):
                                    for f in [5,6,7,8]:
                                        group1value = dfa2[f][(dfa2[grouping[0]]==1)&(one_prob<=dec1[g])&(one_prob>dec1[g-1])].mean()
                                        group2value = dfa2[f][(dfa2[grouping[1]]==1)&(one_prob<=dec1[g])&(one_prob>dec1[g-1])].mean()
                                        difference = group1value - group2value
                                        differences.append(difference)
                                #for the first stratum
                                for f in [5,6,7,8]:
                                    group1value = (dfa2[f][(dfa2[grouping[0]]==1)&(one_prob>dec1[stratify_level-2])]).mean()
                                    group2value = (dfa2[f][(dfa2[grouping[1]]==1)&(one_prob>dec1[stratify_level-2])]).mean()
                                    difference = group1value - group2value
                                    differences.append(difference)
                                #for the last stratum
                                for f in [5,6,7,8]:
                                    group1value = (dfa2[f][(dfa2[grouping[0]]==1)&(one_prob<dec1[0])]).mean()
                                    group2value = (dfa2[f][(dfa2[grouping[1]]==1)&(one_prob<dec1[0])]).mean()
                                    difference = group1value - group2value
                                    differences.append(difference)

                                covariate_diff = np.sum(np.abs(differences))
                                differences2 = []
                                ##############ONE PROB###
                                for g in range(1,stratify_level-2):
                                    for f in [5,6,7,8]:
                                        group1value = one_prob[(dfa2[grouping[0]]==1)&(one_prob<=dec1[g])&(one_prob>dec1[g-1])].mean()
                                        group2value = one_prob[(dfa2[grouping[1]]==1)&(one_prob<=dec1[g])&(one_prob>dec1[g-1])].mean()
                                        difference = group1value - group2value
                                        differences2.append(difference)
                                for f in [5,6,7,8]:
                                    group1value = (one_prob[(dfa2[grouping[0]]==1)&(one_prob>dec1[stratify_level-2])]).mean()
                                    group2value = (one_prob[(dfa2[grouping[1]]==1)&(one_prob>dec1[stratify_level-2])]).mean()
                                    difference = group1value - group2value
                                    differences2.append(difference)
                                for f in [5,6,7,8]:
                                    group1value = (one_prob[(dfa2[grouping[0]]==1)&(one_prob<dec1[0])]).mean()
                                    group2value = (one_prob[(dfa2[grouping[1]]==1)&(one_prob<dec1[0])]).mean()
                                    difference = group1value - group2value
                                    differences2.append(difference)

                                propensity_diff = np.sum(np.abs(differences))

                                ###############
                                unbalanced = []
                                for g in range(1,stratify_level-2):
                                    for f in [5,6,7,8]:
                                        group0 = dfa2[f][(dfa2[grouping[0]]==1)&(one_prob<=dec1[g])&(one_prob>dec1[g-1])]
                                        group1 = dfa2[f][(dfa2[grouping[1]]==1)&(one_prob<=dec1[g])&(one_prob>dec1[g-1])]
                                        if len(group0) < 2 or len(group1) < 2:
                                            pass
                                        else:
                                            sig = ttest(group0,group1)[1]
                                            if sig < .05:
                                                unbalanced.append({'group#':g,
                                                                  'variable#':f,
                                                                  'group0size':len(group0),
                                                                  'group1size':len(group1),
                                                                  'p-value':sig})
                                for f in [5,6,7,8]:
                                    g = stratify_level-1
                                    group0 = (dfa2[f][(dfa2[grouping[0]]==1)&(one_prob>dec1[stratify_level-2])])
                                    group1 = (dfa2[f][(dfa2[grouping[1]]==1)&(one_prob>dec1[stratify_level-2])])
                                    if len(group0) < 2 or len(group1) < 2:
                                        pass
                                    else:
                                        sig = ttest(group0,group1)[1]
                                        if sig < .05:
                                            unbalanced.append({'group#':g,
                                                                  'variable#':f,
                                                                  'group0size':len(group0),
                                                                  'group1size':len(group1),
                                                                  'p-value':sig})
                                    #for the last stratum
                                for f in [5,6,7,8]:
                                    g = 0
                                    group0 = (dfa2[f][(dfa2[grouping[0]]==1)&(one_prob<dec1[0])])
                                    group1 = (dfa2[f][(dfa2[grouping[1]]==1)&(one_prob<dec1[0])])
                                    if len(group0) < 2 or len(group1) < 2:
                                        pass
                                    else:
                                        sig = ttest(group0,group1)[1]
                                        if sig < .05:
                                            unbalanced.append({'group#':g,
                                                                  'variable#':f,
                                                                  'group0size':len(group0),
                                                                  'group1size':len(group1),
                                                                  'p-value':sig})
                                ###############

                                ###
                                counting1 = []
                                counting2 = []
                                for x in range(stratify_level):
                                    counting1.append(len(final_dict_strat['treatment{}'.format(x)]))
                                    counting2.append(len(final_dict_strat['control{}'.format(x)]))


                                #Simple Stratification
                                #using binary cutoff points for covariates (greater than median is set to 1, less than median is set to 0)
                                ###############
                                tmp2[5].median()
                                tmp2[15] = tmp2[5].map(lambda x: 1 if x > tmp2[5].median() else 0)
                                tmp2[16] = tmp2[6].map(lambda x: 1 if x > tmp2[6].median() else 0)
                                tmp2[17] = tmp2[7].map(lambda x: 1 if x > tmp2[7].median() else 0)
                                tmp2[18] = tmp2[8].map(lambda x: 1 if x > tmp2[8].median() else 0)

                                tmp2['group'] = tmp2[15].astype(str) + tmp2[16].astype(str) + tmp2[17].astype(str) + tmp2[18].astype(str)

                                groups2 = ['0000','0001','0010','0011','0100','0101','0110','0111','1000','1001','1010','1011','1100','1101','1110','1111']

                                finalresult = 0
                                for group in groups2:
                                    meantreatment = tmp2[(tmp2.group == group)&(tmp2[grouping[0]] == 1)][4].mean()
                                    meancontrol = tmp2[(tmp2.group == group)&(tmp2[grouping[1]] == 1)][4].mean()
                                    temp = tmp2[(tmp2.group == group)&((tmp2[grouping[1]] == 1)|(tmp2[grouping[0]] == 1))].copy()
                                    groupstd = temp[4].std(ddof=1)
                                    result = (meantreatment - meancontrol) / groupstd

                                    groupmembership = len(temp[4])
                                    total = float(len(tmp2[((tmp2[grouping[1]] == 1)|(tmp2[grouping[0]] == 1))][4]))
                                    weight = groupmembership / total

                                    if result == result and weight == weight:
                                        finalresult += result * weight

                                unbalanced2 = []
                                for g in groups2:
                                    for f in [5,6,7,8]:
                                        group0 = tmp2[(tmp2[grouping[0]]==1)&(tmp2.group == g)][f]
                                        group1 = tmp2[(tmp2[grouping[1]]==1)&(tmp2.group == g)][f]
                                        sig = ttest(group0,group1)[1]
                                        if sig < .05:
                                            unbalanced2.append({'group#':g,
                                                              'variable#':f,
                                                              'group0size':len(group0),
                                                              'group1size':len(group1),
                                                              'p-value':sig})
                                ###############
                                if 'variable#' in pd.DataFrame(unbalanced2).columns:
                                    unbalancedSSvar = pd.DataFrame(unbalanced2)['variable#'].values
                                else:
                                    unbalancedSSvar = None
                                if 'group#' in pd.DataFrame(unbalanced2).columns:
                                    unbalancedSSgroup = pd.DataFrame(unbalanced2)['group#'].values
                                else:
                                    unbalancedSSgroup = None
                                if 'variable#' in pd.DataFrame(unbalanced).columns:
                                    unbalancedCSvar = pd.DataFrame(unbalanced)['variable#'].values
                                else:
                                    unbalancedCSvar = None
                                if 'group#' in pd.DataFrame(unbalanced).columns:
                                    unbalancedCSgroup = pd.DataFrame(unbalanced)['group#'].values
                                else:
                                    unbalancedCSgroup = None

                                #final output
                                final_effect_sizes.append({'Treatment':grouping[0],
                                                           'Control':grouping[1],
                                                           'Proportion':pgroup,
                                                           'N':N,
                                                           'Sim#':z,
                                                           'CovMatrix':tmp2.corr(),
                                                           'CorrDiff':df3.abs().sum().sum(),
                                                           'AvgBeta':avgbeta,
                                                           'CovChange':cov_change,
                                                           'simpleCohenMeasure':simpleCohenMeasure,
                                                           'linear_regression_result':linear_regression_result,
                                                           'weighted_regression_result':weighted_regression_result,
                                                           'stratifiedCohenMeasure':CohenD,
                                                           'meangroup1':meangroup1,
                                                           'meangroup2':meangroup2,
                                                           'stdgroup1':stdgroup1,
                                                           'stdgroup2':stdgroup2,
                                                           'mediangroup1':mediangroup1,
                                                           'mediangroup2':mediangroup2,
                                                           'covariate_diff':covariate_diff,
                                                           'propensity_diff':propensity_diff,
                                                           'counting1':counting1,
                                                           'counting2':counting2,
                                                           'newCorr':newCorr,
                                                           'simplestratification':finalresult,
                                                           'unbalancedSSvar#':unbalancedSSvar,
                                                           'unbalancedSSgroup#':unbalancedSSgroup,
                                                           'unbalancedCSvar#':unbalancedCSvar,
                                                           'unbalancedCSgroup#':unbalancedCSgroup,
                                                           'covariate_elimination':cov_elim
                                                          })
finaltest = pd.DataFrame(final_effect_sizes)
#finaltest.to_csv('{}_{}.csv'.format(z,time.time()),index=False)
end = time.time()
total_time = end-start
# print("Time: " + str(total_time/60) + " minutes")
# print("Correlation Diff: " + str(corr_diff))
# print "Final Effect Size: " + str(final_effect_sizes)

In [14]:
tmp = finaltest[['AvgBeta','Control','Treatment','CovChange','Proportion','simpleCohenMeasure','simplestratification','stratifiedCohenMeasure','weighted_regression_result']]

In [16]:
tmp[(tmp.Control == 3)&(tmp.Treatment == 0)]

,AvgBeta,Control,Treatment,CovChange,Proportion,simpleCohenMeasure,simplestratification,stratifiedCohenMeasure,weighted_regression_result
8,0,3,0,"(0, 0)",0,1.336628,1.321314,1.370916,1.320065
9,0,3,0,"(0, 0)",0,1.336628,1.321314,1.347950,1.356843
16,1,3,0,"(0, 0)",0,1.336628,1.321314,1.343842,1.289228
22,1,3,0,"(0, 0)",0,1.336628,1.321314,1.359434,1.350109
32,0,3,0,"(1, 0)",0,1.336628,1.395899,1.477489,1.455792
33,0,3,0,"(1, 0)",0,1.336628,1.395899,1.427997,1.452227
40,1,3,0,"(1, 0)",0,1.336628,1.395899,1.440211,1.285218
46,1,3,0,"(1, 0)",0,1.336628,1.395899,1.423167,1.354170
56,0,3,0,"(1, -1)",0,1.336628,0.639603,1.505094,1.400228
57,0,3,0,"(1, -1)",0,1.336628,0.639603,1.558843,1.442988


In [9]:
finaltest[(finaltest.Control == 3)&(finaltest.Treatment == 0)].groupby('N').mean()[['linear_regression_result','weighted_regression_result','simpleCohenMeasure','simplestratification','stratifiedCohenMeasure']]

,linear_regression_result,weighted_regression_result,simpleCohenMeasure,simplestratification,stratifiedCohenMeasure
N,,,,,
1000,1.348091,1.393301,1.412611,1.406305,1.432053
3000,1.405564,1.466006,1.467262,1.491562,1.491335
5000,1.409116,1.477580,1.474114,1.493984,1.496227
10000,1.449706,1.518674,1.502679,1.526781,1.534849
20000,1.439679,1.513157,1.504900,1.534952,1.537213


In [ ]:
#TODO for simulation
#Run 1000 iterations of sample sizes 1000, 5000, 10000, 30000, 50000
#Run linear regression to get coefficient (for each group comparison)
#Average all coefficients (across the 1000 iterations) to get true value

#Compare proportion change because actual value group mean differences change when proportion changes (it's my hypothesis)
#Once tested, throw out proportion change

#Run 1000 iterations on supercomputer with everything (try to distribute as much as possible)
#Plug in true values for bias equations
#Analyze with your analysis script and meet with Ross to make sure everything looks ok